In [1]:
import pandas as pd
import zipfile
import json
import requests

## Create Necessary Data Files

In [2]:
path = '../data/'
fname = 'CTA_ORIGINAL.csv.zip'
fnames = ['cta_unique_stations.csv', 'income.csv', 'pct_college.csv', 'pct_white_pop2010.csv', 'pct_white_pop2020.csv']
state = '17' ## Illinois
county = '031' ## Cook County

## Income ##
url = 'https://api.census.gov/data/2019/acs/acs5/subject?get=S1901_C01_012E&for=tract:*&in=state:{}%20county:{}'.format(state,county)
json_req = requests.get(url).json()
income = pd.DataFrame(json_req[1:], columns = json_req[0])
income['geoID'] = income['state']+income['county']+income['tract']
income = income.rename(columns={'S1901_C01_012E': 'medianIncome'})
income = income.drop(labels=['state', 'county', 'tract'],axis=1)

## Educational Attainment ##
url = 'https://api.census.gov/data/2019/acs/acs5/subject?get=S1501_C01_006E,S1501_C01_015E&for=tract:*&in=state:{}%20county:{}'.format(state,county)
json_req = requests.get(url).json()
education = pd.DataFrame(json_req[1:], columns = json_req[0])
education['geoID'] = education['state']+education['county']+education['tract']
education = education.rename(columns={'S1501_C01_006E': 'total25PlusPop', 'S1501_C01_015E': 'BachelorPlus'})
education = education.drop(labels=['state', 'county', 'tract'],axis=1)

## Percent White Alone 2010 ##
url = 'https://api.census.gov/data/2010/dec/sf1?get=P008001,P008003&for=tract:*&in=state:{}%20county:{}'.format(state,county)
json_req = requests.get(url).json()
pctWhite2010 = pd.DataFrame(json_req[1:], columns = json_req[0])
pctWhite2010['geoID'] = pctWhite2010['state']+pctWhite2010['county']+pctWhite2010['tract']
pctWhite2010 = pctWhite2010.rename(columns={'P008001': 'total', 'P008003': 'whiteAlone'})
pctWhite2010 = pctWhite2010.drop(labels=['state', 'county', 'tract'],axis=1)

## Percent White Alone 2020 ##
url = 'https://api.census.gov/data/2020/dec/pl?get=P1_001N,P1_003N&for=tract:*&in=state:{}%20county:{}'.format(state,county)
json_req = requests.get(url).json()
pctWhite2020 = pd.DataFrame(json_req[1:], columns = json_req[0])
pctWhite2020['geoID'] = pctWhite2020['state']+pctWhite2020['county']+pctWhite2020['tract']
pctWhite2020 = pctWhite2020.rename(columns={'P1_001N': 'total', 'P1_003N': 'whiteAlone'})
pctWhite2020 = pctWhite2020.drop(labels=['state', 'county', 'tract'],axis=1)

income.to_csv(path+'income.csv', index=False)
education.to_csv(path+'pct_college.csv',index=False)
pctWhite2010.to_csv(path+'pct_white_pop2010.csv',index=False)
pctWhite2020.to_csv(path+'pct_white_pop2020.csv',index=False)

## Load in Data Files

In [3]:
## Unzip DataFile ##
with zipfile.ZipFile(path+fname,"r") as zip_ref:
    zip_ref.extractall(path)
rides = pd.read_csv(path+fname[:-4])

## Create DataFrames for other files ##
dataframes = {}
for fname in fnames:
    dataframes[fname[:-4]] = pd.read_csv(path+fname)

## Link Stations to Census Blocks

In [4]:
## Link Stations to Census Blocks ##
def find_geoid2010(lat, long):
    url = 'https://geo.fcc.gov/api/census/area?lat=' + str(lat)  + '&lon=' + str(long) + '&censusYear=2010&format=json'
    json_req = requests.get(url).json()
    return int(json_req['results'][0]['block_fips'][:-4]) ## [:-4] converts between fips and geoID

def find_geoid2020(lat, long):
    url = 'https://geo.fcc.gov/api/census/area?lat=' + str(lat)  + '&lon=' + str(long) + '&censusYear=2020&format=json'
    json_req = requests.get(url).json()
    return int(json_req['results'][0]['block_fips'][:-4]) ## [:-4] converts between fips and geoID

dataframes['cta_unique_stations']['geoID2010'] = dataframes['cta_unique_stations'].apply(lambda row: find_geoid2010(row['LAT'],row['LONG']), axis=1)
dataframes['cta_unique_stations']['geoID2020'] = dataframes['cta_unique_stations'].apply(lambda row: find_geoid2020(row['LAT'],row['LONG']), axis=1)
dataframes['cta_unique_stations']['geoID2010'] = pd.to_numeric(dataframes['cta_unique_stations']['geoID2010'])
dataframes['cta_unique_stations']['geoID2020'] = pd.to_numeric(dataframes['cta_unique_stations']['geoID2020'])

In [5]:
## Convert Between FIPS and GEOID ##
dataframes['cta_unique_stations'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   STOP_ID                   143 non-null    int64  
 1   DIRECTION_ID              143 non-null    object 
 2   STOP_NAME                 143 non-null    object 
 3   STATION_NAME              143 non-null    object 
 4   STATION_DESCRIPTIVE_NAME  143 non-null    object 
 5   MAP_ID                    143 non-null    int64  
 6   ADA                       143 non-null    bool   
 7   RED                       143 non-null    bool   
 8   BLUE                      143 non-null    bool   
 9   G                         143 non-null    bool   
 10  BRN                       143 non-null    bool   
 11  P                         143 non-null    bool   
 12  Pexp                      143 non-null    bool   
 13  Y                         143 non-null    bool   
 14  Pnk       

## Link Stations to Income, Education Attainment, and White Population

In [6]:
fnames = ['income', 'pct_college']
for fname in fnames:
    dataframes['cta_unique_stations'] = dataframes['cta_unique_stations'].merge(dataframes[fname],how= 'inner', left_on='geoID2010', suffixes=('','_TRASH'),right_on='geoID')

In [7]:
df2010 = dataframes['cta_unique_stations'].merge(dataframes['pct_white_pop2010'],how='inner',on='geoID')
df2020 = dataframes['cta_unique_stations'].merge(dataframes['pct_white_pop2020'],how='inner',on='geoID')
final = df2010.merge(df2020, how ='left', on='STOP_ID',suffixes=('','TRASH')).rename(columns={'whiteAloneTRASH': 'whiteAlone2', 'totalTRASH': 'total2'})
final = final.drop(final.columns[final.columns.str.contains('TRASH$')], axis=1)
final = final.drop('geoID', axis=1)

final['whiteAloneFinal'] = final.apply(lambda row: row['whiteAlone2'] if not row['whiteAlone2'] else row['whiteAlone'],axis=1)
final['totalFinal'] = final.apply(lambda row: row['total2'] if not row['total2'] else row['total'],axis=1)
final = final.drop(['whiteAlone','whiteAlone2','total','total2'], axis=1)
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143 entries, 0 to 142
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   STOP_ID                   143 non-null    int64  
 1   DIRECTION_ID              143 non-null    object 
 2   STOP_NAME                 143 non-null    object 
 3   STATION_NAME              143 non-null    object 
 4   STATION_DESCRIPTIVE_NAME  143 non-null    object 
 5   MAP_ID                    143 non-null    int64  
 6   ADA                       143 non-null    bool   
 7   RED                       143 non-null    bool   
 8   BLUE                      143 non-null    bool   
 9   G                         143 non-null    bool   
 10  BRN                       143 non-null    bool   
 11  P                         143 non-null    bool   
 12  Pexp                      143 non-null    bool   
 13  Y                         143 non-null    bool   
 14  Pnk       

## Output to CSV

In [8]:
final.to_csv(path+'final_station_data.csv',index=False)